In [43]:
import pandas as pd

from Project.Database import Db
from optimisation_problem import hourly_house_df, slice_emission_vector, power_consumption_vector, NZERTF_optimiser, SE_time_df, json_to_dataframe

In [44]:
def find_emissions(df, emission_vec, day_number):
    energy_vec = df.loc[lambda self: self['Day'] == day_number, 'Consumption'].reset_index(drop=True)
    energy_consumed = energy_vec.multiply(emission_vec)
    for index, emission in energy_consumed[lambda self: self > 0].iteritems():
        df.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), 'Emission'] += emission
    return df

In [45]:
meta = Db.load_data(meta=True, hourly=False, year=2, consumption=False).loc[
    lambda self: (~self['Consumer_Match'].isna()), 'Consumer_Match']

movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher',
                      'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

appliances = ['Timestamp'] + meta.index.tolist() + meta.tolist()

patterns = SE_time_df(
    dataframe=json_to_dataframe(
        year=2,
        level=1,
        exclude_follows=True,
        with_redundancy=False).loc[lambda self: self['pattern'].isin(movable_appliances)],
    TAT=0.1,
    LS_quantile=0.9)

power_consumption = power_consumption_vector(movable_appliances=movable_appliances)

# with redundancy <- w.r
# without redundancy <- w.o.r
# movable appliances unoptimised <- m.a.u.o
# movable appliances optimised <- m.a.o
NZERTF_optimisation = {
    'w.r': Db.load_data(year=2, hourly=False)[appliances],
    'w.o.r': Db.load_data(year=2, hourly=False, with_redundancy=False)[appliances]
}

NZERTF_optimisation.update({
    'm.a.u.o': NZERTF_optimisation['w.o.r'].copy()[['Timestamp'] + movable_appliances]
})

timestamps = pd.to_datetime(
    NZERTF_optimisation['w.r']['Timestamp'].dt.strftime('%Y-%m-%d %H').unique(),
    format='%Y-%m-%d %H')

production = Db.load_data(
    consumption=False,
    production=True,
    year=2)['CO2(Grams)/kWh'][lambda self: self.index.isin(timestamps)]
production = production.groupby(pd.to_datetime(production.index.strftime('%Y-%m-%d %H'))).sum()

for key in NZERTF_optimisation.keys():
    if all([att in NZERTF_optimisation[key].columns for att in meta.tolist()]):
        NZERTF_optimisation.update({
            key: pd.concat(
                objs=(hourly_house_df(house_df=NZERTF_optimisation[key][['Timestamp'] + meta.index.tolist()].copy(),
                                      aggregate_func='mean'),
                      hourly_house_df(house_df=NZERTF_optimisation[key][['Timestamp'] + meta.tolist()].copy(),
                                      aggregate_func='sum')[meta.tolist()]),
                axis=1)
        })
        NZERTF_optimisation[key]['Consumption'] = NZERTF_optimisation[key][meta.tolist()].sum(1).div(60_000)
        NZERTF_optimisation[key].drop(labels=meta.tolist(), inplace=True, axis=1)
    else:
        switch_rows = {}
        for app in movable_appliances:
            switch_rows.update({
                app: NZERTF_optimisation[key].copy().loc[lambda self: (self[app] != self[app].shift(1)) & (self[app] == 1)]
            })
            switch_rows[app]['Day'] = switch_rows[app]['Timestamp'].dt.dayofyear
            switch_rows[app]['Day'] = switch_rows[app]['Day'] - switch_rows[app].loc[lambda self: self.index[0], 'Day']

            first_index = switch_rows[app].loc[lambda self: self['Day'] < 0].index.tolist()[0]
            switch_rows[app].loc[first_index:, 'Day'] = switch_rows[app].loc[first_index:, 'Day'] + 365

            switch_rows[app]['Hour'] = switch_rows[app]['Timestamp'].dt.hour

        NZERTF_optimisation.update({
            key: hourly_house_df(house_df=NZERTF_optimisation[key], aggregate_func='max')
        })

        for app in movable_appliances:
            timespan = patterns[app]['Timespan'][0] / 60
            NZERTF_optimisation[key][app] = 0
            for row in switch_rows[app].itertuples():
                remaining = timespan
                hour = row.Hour
                while remaining > 0:
                    current = NZERTF_optimisation[key].loc[
                        (lambda self: (self['Day'] == row.Day) & (self['Hour'] == hour)), app].values[0]
                    print(current)
                    NZERTF_optimisation[key].loc[
                        (lambda self: (self['Day'] == row.Day) & (self['Hour'] == hour)), app] = min(current + remaining, 1)
                    remaining = max(current + remaining - 1, 0)
                    hour += 1

        NZERTF_optimisation[key]['Consumption'] = NZERTF_optimisation[key][movable_appliances].dot(
            power_consumption).div(1_000)

    for day in NZERTF_optimisation[key]['Day'].unique():
        NZERTF_optimisation[key] = find_emissions(
            df=NZERTF_optimisation[key],
            emission_vec=slice_emission_vector(
                production_vectors=production,
                day_number=day), day_number=day)

NZERTF_optimisation.update({
    'm.a.o': NZERTF_optimiser()
})

NZERTF_emission = {}

for key in NZERTF_optimisation.keys():
    NZERTF_emission.update({
        key: NZERTF_optimisation[key]['Emission'].sum()
    })

0
0
0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
for key, value in NZERTF_optimisation.items():
    print(f'{key}: {value["Emission"]}')

w.r: 0        25481.390734
1        26419.462216
2        26218.277271
3        26276.909272
4        26287.695996
            ...      
8667    195494.914719
8668    175027.237188
8669    167321.565535
8670    213162.462765
8671     88438.888401
Name: Emission, Length: 8672, dtype: float64
w.o.r: 0        25481.390734
1        26419.462216
2        26218.277271
3        26276.909272
4        26287.695996
            ...      
8667    159266.721655
8668    158688.985590
8669    167321.565535
8670    212525.033626
8671     87796.668878
Name: Emission, Length: 8672, dtype: float64
m.a.u.o: 0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
           ...    
8667      0.000000
8668    217.245498
8669    217.245498
8670     47.069858
8671      0.000000
Name: Emission, Length: 8672, dtype: float64
m.a.o: 0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.000000
           ...     
8667       0.000000
8668

In [ ]:
NZERTF_emission

{'w.r': 629579158.1547395,
 'w.o.r': 611331281.5435189,
 'm.a.u.o': 557131.2715596153,
 'm.a.o': 582924.825771024}

In [ ]:
redundancy_reduction = NZERTF_emission['w.r'] - NZERTF_emission['w.o.r']
optimisation_reduction = NZERTF_emission['m.a.u.o'] - NZERTF_emission['m.a.o']
total_reduction = redundancy_reduction + optimisation_reduction

In [ ]:
redundancy_reduction

18247876.6112206

In [ ]:
optimisation_reduction

-25793.554211408715

In [ ]:
total_reduction

18222083.05700919

In [ ]:
NZERTF_optimisation['m.a.o']

,Day,Hour,Timestamp,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission,CumulativeEmission
0,0,0,2015-02-01 00:00:00,0,0,0,0,0,0.000000,0.000000
1,0,1,2015-02-01 01:00:00,0,0,0,0,0,0.000000,0.000000
2,0,2,2015-02-01 02:00:00,0,0,0,0,0,0.000000,0.000000
3,0,3,2015-02-01 03:00:00,0,0,0,0,0,0.000000,0.000000
4,0,4,2015-02-01 04:00:00,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
8667,364,19,2016-01-31 19:00:00,0,0,0,0,0,0.000000,581560.867193
8668,364,20,2016-01-31 20:00:00,0,0,1,0,0,58.557385,581619.424577
8669,364,21,2016-01-31 21:00:00,1,0,1,0,0,1161.117351,582780.541928
8670,364,22,2016-01-31 22:00:00,1,0,0,0,0,118.589460,582899.131388


In [ ]:
NZERTF_optimisation['m.a.u.o']

,Day,Hour,Timestamp,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission,Consumption
0,0,0,2015-02-01 00:00:00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
1,0,1,2015-02-01 01:00:00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
2,0,2,2015-02-01 02:00:00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
3,0,3,2015-02-01 03:00:00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
4,0,4,2015-02-01 04:00:00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...
8667,364,19,2016-01-31 19:00:00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0000
8668,364,20,2016-01-31 20:00:00,1.000000,0.0,0.0,0.0,0.0,217.245498,0.2820
8669,364,21,2016-01-31 21:00:00,1.000000,0.0,0.0,0.0,0.0,217.245498,0.2820
8670,364,22,2016-01-31 22:00:00,0.216667,0.0,0.0,0.0,0.0,47.069858,0.0611


In [ ]:
patterns

{'Load_StatusClothesWasher':     TotalAbsSupport  AbsSupport  EventCount  ExternalUtility  RelSupport  \
 0                 0           0         312         0.427397    0.000000   
 1                 0           0         312         0.427397    0.000000   
 2                 0           0         312         0.427397    0.000000   
 3                 0           0         312         0.427397    0.000000   
 4                 0           0         312         0.427397    0.000000   
 5                 0           0         312         0.427397    0.000000   
 6                 0           0         312         0.427397    0.000000   
 7                 0           0         312         0.427397    0.000000   
 8                 0           0         312         0.427397    0.000000   
 9                 0           0         312         0.427397    0.000000   
 10                1           1         312         0.427397    0.002740   
 11              103         103         312    